In [ ]:
! pip install meta-ai-api >  inst
! pip install nltk > inst

In [ ]:
! pip install meta-ai-api >  inst
! pip install nltk > inst

from meta_ai_api import MetaAI

import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
import time 
import re


ai = MetaAI()


def analyze_sentences(sentences):
  response = ai.prompt(message=f"""help me to analyze texts as assistant for TOEFL student mentor
  I will provide you the sentences and you should Analyze it grammatically.
  You must write the word itself between span tag with class "tooltip" and the analyze in the between  span tag with class "tooltiptext" and also add reduced words too with class "reduced" in span tag
  also emphisize about the Reductions, conditionals, Inversion, Clauses, Sentences type
  if my request was like this:
  In the wake of the Roman Empire's conquest of Britain in the first century A.D., a large number of troops stayed in the new province, and these troops had a considerable impact on Britain with their camps, fortifications, and participation in the local economy.
  your response should be like this:
  <span class="tooltip">In the wake of<span class="tooltiptext">prepositional phrase indicating a time period</span></span> <span class="tooltip">the Roman Empire's conquest of Britain<span class="tooltiptext">noun phrase functioning as the object of the preposition "of"</span></span> <span class="tooltip">in the first century A.D.<span class="tooltiptext">adverbial phrase indicating time</span></span> <span class="tooltip">, a large number of troops<span class="tooltiptext">noun phrase and the subject of the sentence</span></span> <span class="tooltip">stayed<span class="tooltiptext">verb in the past tense</span></span> <span class="tooltip">in the new province<span class="tooltiptext">prepositional phrase indicating location</span></span> <span class="tooltip">, and<span class="tooltiptext">coordinating conjunction connecting two independent clauses</span></span> <span class="tooltip">these troops<span class="tooltiptext">pronoun referring back to "a large number of troops"</span></span> <span class="tooltip">had<span class="tooltiptext">verb in the past tense</span></span> <span class="tooltip">a considerable impact<span class="tooltiptext">noun phrase and the object of the verb "had"</span></span> <span class="tooltip">on Britain<span class="tooltiptext">prepositional phrase indicating location</span></span> <span class="tooltip">with their camps, fortifications, and participation in the local economy<span class="tooltiptext">prepositional phrase indicating the means by which the troops had an impact</span></span> <span class="tooltip">.
  Now do it for this sentences:
  {' '.join(sentences)}
                       
  please dont write any other words in respone jsut the html code. I need unique format as I told.
  """)
  return response['message']


def analyze_whole_text(text, step=1):

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Print the list of sentences
    print(sentences)


    responses = []
    i = 0
    while i < len(sentences):
        # Get the next two sentences
        current_batch = sentences[i:i+step]
        if len(current_batch) < step:
            break  # If less than step sentences remain, break the loop
        
        # Analyze the current batch of sentences
        response = analyze_sentences(current_batch)

        time.sleep(30)
        
        # Check if the sentences are in the response and remove them if they are
        if all(sentence in response for sentence in current_batch):
            sentences = [s for s in sentences if s not in current_batch]
        else:
            i += step  # Move to the next pair of sentences
        responses.append(response)
    return responses  # Return the last response or modify as needed


def convert_to_html_file_with_tooltips_modified(title, text):

    # Regex pattern to find text followed by annotations inside square brackets
    pattern = r'([^[]*)\[(.*?)\]'
    
    # Function to wrap matched text in a span with a title for tooltip
    def replace_with_tooltip(match):
        # Extracting the text before and inside the brackets
        text_before_brackets = match.group(1).strip()
        content_inside_brackets = match.group(2).strip()
        # Replacing the matched text with a tooltip span element
        return f'<span class="tooltip">{text_before_brackets}<span class="tooltiptext">{content_inside_brackets}</span></span> '

    # Replacing all occurrences in the text
    tooltip_text = re.sub(pattern, replace_with_tooltip, text)

    # HTML template
    html_template = f"""<!DOCTYPE html>
<html>
<head>
<style>
body {{
  text-align: left;
  padding: 20px;
}}

.tooltip {{
  position: relative;
  display: inline-block;
  border-bottom: 1px dotted black;
  margin-right: 5px; /* Adding space between tooltip groups */
}}

.tooltip .tooltiptext {{
  visibility: hidden;
  width: 160px; /* Wider to avoid overlap */
  background-color: black;
  color: #fff;
  text-align: center;
  border-radius: 6px;
  padding: 5px;

  /* Position the tooltip */
  position: absolute;
  z-index: 1;
  left: 50%;
  transform: translateX(-50%);
  bottom: 100%;
  margin-bottom: 5px; /* Space between text and tooltip */
}}

.tooltip:hover .tooltiptext {{
  visibility: visible;
}}
</style>
</head>
<body>

<h2>{title}</h2>

<div>{tooltip_text}</div>

</body>
</html>
"""

    # Writing to an HTML file
    with open("Reading.html", "w") as file:
        file.write(html_template)
    return "./Reading.html"


text = """
In the wake of the Roman Empire's conquest of Britain in the first century A.D., a large number of troops stayed in the new province, and these troops had a considerable impact on Britain with their camps, fortifications, and participation in the local economy. Assessing the impact of the army on the civilian population starts from the realization that the soldiers were always unevenly distributed across the country. Areas rapidly incorporated into the empire were not long affected by the military. Where the army remained stationed, its presence was much more influential. The imposition of a military base involved the requisition of native lands for both the fort and the territory needed to feed and exercise the soldiers' animals. The imposition of military rule also robbed local leaders of opportunities to participate in local government, so social development was stunted and the seeds of disaffection sown. This then meant that the military had to remain to suppress rebellion and organize government.

Economic exchange was clearly very important as the Roman army brought with it very substantial spending power. Locally with respect to a particular place or situation, a fort had two kinds of impact. Its large population needed food and other supplies. Some of these were certainly brought from long distances, but demands were inevitably placed on the local area. Although goods could be requisitioned, they were usually paid for, and this probably stimulated changes in the local economy. When not campaigning, soldiers needed to be occupied; otherwise, they represented a potentially dangerous source of friction and disloyalty. Hence a writing tablet dated 25 April tells of 343 men at one fort engaged on tasks like shoemaking, building a bathhouse, operating kilns, digging clay, and working lead. Such activities had a major effect on the local area, in particular with the construction of infrastructure such as roads, which improved access to remote areas.

Each soldier received his pay, but in regions without a developed economy, there was initially little on which it could be spent. The pool of excess cash rapidly stimulated a thriving economy outside fort gates. Some of the demand for the services and goods was no doubt fulfilled by people drawn from far afield, but some local people certainly became entwined in this new economy. There was informal marriage with soldiers, who until A.D. 197 were not legally entitled to wed, and whole new communities grew up near the forts. These settlements acted like small towns, becoming centers for the artisan and trading populations.

The army also provided a mean of personal advancement for auxiliary soldiers recruited from the native peoples, as a man obtained hereditary Roman citizenship on retirement after service in an auxiliary regiment. Such units recruited on an ad hoc (as needed) basis from the area in which they were stationed, and there was evidently large-scale recruitment within Britain. The total numbers were at least 12,500 men up to the reign of the emperor Hadrian (A.D. 117-138), with a peak around A.D. 80. Although a small proportion of the total population, this perhaps had a massive local impact when a large proportion of the young men were removed from an area. Newly raised regiments were normally transferred to another province from whence it was unlikely that individual recruits would ever return. Most units raised in Britain went elsewhere on the European continent, although one is recorded in Morocco. The reverse process brought young men to Britain, where many continued to live after their 20 to 25 years of service, and this added to the cosmopolitan Roman character of the frontier population. By the later Roman period, frontier garrisons (groups of soldiers) were only rarely transferred, service in units became effectively hereditary, and forts were no longer populated or maintained at full strength.

This process of settling in as a community over several generations, combined with local recruitment, presumably accounts for the apparent stability of the British northern frontier in the later Roman period. It also explains why some of the forts continued in occupation long after Rome ceased to have any formal authority in Britain, at the beginning of the fifth century A.D. The circumstances that had allowed natives to become Romanized also led the self-sustaining military community of the frontier area to become effectively British.
"""
result = analyze_whole_text(text)

convert_to_html_file_with_tooltips_modified("The Roman Army's Impact on Britain", '<br>'.join(result))
 